# Program to add data to mySQL table

In [1]:
import pandas as pd
data1 = pd.read_csv("Data/WebApp_Final_Data.csv")
data1.columns = ["cname","cnum","iid","amount","duedate","preddate"]
data1

,cname,cnum,iid,amount,duedate,preddate
0,LOB llc,0140104409,2.960617e+09,76241.35,2020-03-19,2020-03-13
1,BJ'S foundation,0200726979,1.930572e+09,362.81,2020-03-13,2020-03-13
2,GLA,0200742521,1.930567e+09,23155.48,2020-03-13,2020-03-13
3,COST llc,0200794332,1.930574e+09,12610.49,2020-03-13,2020-03-12
4,US in,0200357714,1.930576e+09,70869.05,2020-03-13,2020-03-13
...,...,...,...,...,...,...
9994,GODL us,100012181,1.930897e+09,120.00,2020-06-12,2020-06-01
9995,ANTONIO llc,0200447006,1.930893e+09,31714.10,2020-05-28,2020-05-28
9996,FLANAG trust,0140104392,2.960634e+09,5607.95,2020-05-27,2020-05-30
9997,GROC,0200915438,1.930915e+09,8326.87,2020-06-03,2020-06-03


In [3]:
# iid has ".0" in the end, this is to remove it.
data1['iid'] = data1['iid'].astype("str")
data1['iid'] = data1['iid'].str.slice(0,-2,1) 

#Only unique iid's please!
data1.drop_duplicates(subset=['iid'],keep='first',inplace=True)
data1.shape

(9667, 6)

In [54]:
# Save to CSV file
#data1.to_csv('Data/WebApp_Final_Data_unique.csv',index=None)

In [4]:
import mysql.connector

In [5]:
db2 = mysql.connector.connect(host = 'localhost',user='root',password = 'root',auth_plugin='mysql_native_password')
print(db2)

In [6]:
cursor = db2.cursor()
#use a db called highradius.
cursor.execute("use highradius;")

#Purge old data if exists:
cursor.execute("truncate table invdata_test;")
db2.commit()

In [9]:
with open("Data/WebApp_Final_Data_unique.csv","r") as csv:
    lines = csv.readlines()
    count = 0
    for line in lines:
        values = line.strip().split(",")
        cname = '\"'+values[0].strip()+'\"'
        cnum = '\"'+values[1].strip()+'\"'
        cid = '\"'+values[2]+'\"'
        amount = float(values[3])
        ddate = values[4].strip()
        pdate = values[5].strip()

        query = "insert into invdata_test(cname,cnum,iid,amount,duedate,preddate) values({},{},{},{},DATE'{}',DATE'{}')".format(cname,cnum,cid,amount,ddate,pdate)
        print(count,"  ",query)
        cursor.execute(query)
        count+=1
        if(count%1000):
            db2.commit()
            print("Committed the last 1k lines")
        if not line or count>10:
            print("Lines ran out, ending and commiting!")
            db2.commit()
            break
db2.commit()
print(count,"Queries executed. Check table in SQLyog!")

0    insert into invdata_test(cname,cnum,iid,amount,duedate,preddate) values("LOB llc","0140104409","2960616692",76241.35,DATE'2020-03-19',DATE'2020-03-13')
Committed the last 1k lines
1    insert into invdata_test(cname,cnum,iid,amount,duedate,preddate) values("BJ'S  foundation","0200726979","1930572462",362.81,DATE'2020-03-13',DATE'2020-03-13')
Committed the last 1k lines
2    insert into invdata_test(cname,cnum,iid,amount,duedate,preddate) values("GLA","0200742521","1930567164",23155.48,DATE'2020-03-13',DATE'2020-03-13')
Committed the last 1k lines
3    insert into invdata_test(cname,cnum,iid,amount,duedate,preddate) values("COST llc","0200794332","1930573750",12610.49,DATE'2020-03-13',DATE'2020-03-12')
Committed the last 1k lines
4    insert into invdata_test(cname,cnum,iid,amount,duedate,preddate) values("US  in","0200357714","1930576008",70869.05,DATE'2020-03-13',DATE'2020-03-13')
Committed the last 1k lines
5    insert into invdata_test(cname,cnum,iid,amount,duedate,preddate) va

In [10]:
db2.close()